# Example: Using `IntelELM` for Building a House Price Predictor

* Install `IntelELM` library

In [1]:
!python -m pip install intelelm==1.0.3

* Import libraries

In [2]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split
from intelelm import ElmRegressor, MhaElmRegressor
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline

## Loading California Housing Dataset

In [3]:
california_housing = datasets.fetch_california_housing(as_frame=True)

## Dataset: Train/Test Split & Preprocessing

* We choose some features for training our model.

In [4]:
features = ["AveRooms", "AveBedrms", "AveOccup", "Population"]

data = california_housing.frame[features].values

* Train/test split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(data, california_housing.target.values, test_size=0.3, shuffle=True, random_state=1)

* Feature scaling with `MinMaxScaler`

In [6]:
min_max_scaler = MinMaxScaler()
min_max_scaler.fit(X_train)
X_train_scaled = min_max_scaler.transform(X_train)
X_test_scaled = min_max_scaler.transform(X_test)

## Regression with Standard ELM Model

* Tran an `ElmRegressor`

In [7]:
elm_regressor = ElmRegressor(hidden_size=100, act_name="relu")
elm_regressor.fit(X_train_scaled, y_train)

ElmRegressor(act_name='relu', hidden_size=100)

In [8]:
elm_predicted = elm_regressor.predict(X_test_scaled)

* Report regression results

In [9]:
print("RMSE:", metrics.mean_squared_error(y_test, elm_predicted, squared=False))
print("MAE:", metrics.mean_absolute_error(y_test, elm_predicted))

RMSE: 1.0660753353991896
MAE: 0.8407205484213833


## Regression with `MhaElmRegressor`

* Train a model

In [10]:
opt_params = {
    "name": "GA",
    "epoch": 10,
    "pop_size": 30,
}

mha_elm_regressor = MhaElmRegressor(hidden_size=100, act_name="relu", obj_name="RMSE", optimizer="BaseGA", optimizer_paras=opt_params, verbose=True)


In [11]:
mha_elm_regressor.fit(X_train_scaled, y_train)

2023/10/05 03:42:52 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: Solving single objective optimization problem.


2023/10/05 03:42:58 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 1, Current best: 0.99913, Global best: 0.99913, Runtime: 2.93316 seconds
2023/10/05 03:43:01 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 2, Current best: 1.003097, Global best: 0.99913, Runtime: 3.18233 seconds
2023/10/05 03:43:04 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 3, Current best: 1.00104, Global best: 0.99913, Runtime: 3.12306 seconds
2023/10/05 03:43:07 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 4, Current best: 0.984083, Global best: 0.984083, Runtime: 3.08050 seconds
2023/10/05 03:43:10 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 5, Current best: 0.98369, Global best: 0.98369, Runtime: 3.03170 seconds
2023/10/05 03:43:14 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >Problem: P, Epoch: 6, Current best: 0.983328, Global best: 0.983328, Runtime: 3.41910 seconds
2023/10/05 03:43:17 PM, INFO, mealp

MhaElmRegressor(act_name='relu', hidden_size=100, obj_name='RMSE',
                optimizer=<mealpy.evolutionary_based.GA.BaseGA object at 0x7fe0da943430>,
                optimizer_paras={'epoch': 10, 'name': 'GA', 'pop_size': 30},
                verbose=True)

In [12]:
mha_elm_predicted = mha_elm_regressor.predict(X_test_scaled)

* Report regression results

In [13]:
print("RMSE:", metrics.mean_squared_error(y_test, mha_elm_predicted, squared=False))
print("MAE:", metrics.mean_absolute_error(y_test, mha_elm_predicted))

RMSE: 1.0555342777265628
MAE: 0.760976160243756


* It can be seen that using meta-heuristics based ELM reduces both RSME and MAE errors.